# AutoClassを利用して事前学習済みインスタンスをロードする

**※説明の読みやすさのために，警告を非表示にします．実際のプログラムでは警告の表示を推奨します．**

In [1]:
# 警告の表示を停止
import warnings
warnings.filterwarnings('ignore')
# Transformersのログを非表示
from transformers import logging
logging.set_verbosity_error()

Transformerをベースにした非常に多様なアーキテクチャが存在するため，目的のタスクに合ったモデルを作成するのが難しい場合があります．
Hugging Face Transformersの基本方針として，ライブラリを使いやすく，シンプルで柔軟にするために，`AutoClass`は与えられたチェックポイントから適切なアーキテクチャを自動的に推論し，ロードします．
`from_pretrained()`メソッドを使用すると，事前学習済みモデルを素早くロードできるため，モデルをゼロから学習するための時間やリソースを節約することができます．
このようにチェックポイントに依存しないコードを生成することは，コードがあるチェックポイントで動作すれば，異なるアーキテクチャのモデルでも同じタスクに向けて学習されていれば利用できることを意味します．

> **アーキテクチャ**とはモデルの**構造**を指します．チェックポイントは特定のアーキテクチャの重みです．
> 例えば，`BERT`はアーキテクチャであり，`google-bert/bert-base-uncased`はチェックポイントです．
> モデルはアーキテクチャまたはチェックポイントのどちらかを指す一般的な用語です．


## AutoTokenizer

ほとんどのNLP（自然言語処理）タスクはトークナイザで始まります．
トークナイザは入力をモデルで処理できる形式に変換します．

`AutoTokenizer.from_pretrained()`を使用してトークナイザをロードします．

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

次に，トークナイザに入力を渡します．

In [3]:
sequence = "In a hole in the ground there lived a hobbit."
print(tokenizer(sequence))

{'input_ids': [101, 1999, 1037, 4920, 1999, 1996, 2598, 2045, 2973, 1037, 7570, 10322, 4183, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


## AutoImageProcessor

ビジョンタスクの場合，画像処理が画像を正しい入力形式に変換します．

In [4]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")


# 画像の読み込み
# https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg をPIL Imageとして読み込む
from PIL import Image
import requests
from io import BytesIO

image_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
image = Image.open(BytesIO(requests.get(image_url).content))

# 実際の特長抽出
inputs = image_processor(images=image)

## AutoFeatureExtractor

オーディオタスクの場合，特徴量抽出がオーディオ信号を正しい入力形式に変換します．

`AutoFeatureExtractor.from_pretrained()`を使用して特徴量抽出器をロードします．

In [5]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(
    "ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"
)

# 音声ファイルの読み込み
import torchaudio
audio_url = "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac"
waveform, sample_rate = torchaudio.load(audio_url)

# サンプリングレートの調整
if sample_rate != feature_extractor.sampling_rate:
    resampler = torchaudio.transforms.Resample(sample_rate, feature_extractor.sampling_rate)
    waveform = resampler(waveform)

# 実際の特徴抽出
inputs = feature_extractor(waveform, sampling_rate=feature_extractor.sampling_rate, return_tensors="pt")


## AutoProcessor

マルチモーダルタスクの場合，2つの前処理ツールを組み合わせることが必要になってきます．
例えば，`LayoutLMV2`モデルは画像を処理するための画像処理と，テキストを処理するためのトークナイザが必要です．
`AutoProcessor`は，これらの両方を組み合わせます．


In [6]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("microsoft/layoutlmv2-base-uncased")
inputs = processor(image, "In a hole in the ground there lived a hobbit.")


## AutoModel

`AutoModelFor???`クラスは特定のタスクに対して事前学習済モデルをロードできます．
使用できるモデルの一覧は[こちら](https://huggingface.co/docs/transformers/ja/model_doc/auto)を参照してください．

例えば，`AutoModelForSequenceClassification.from_pretrained()`を使用して，シーケンス分類用のモデルをロードできます．


In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")

同じチェックポイントを再利用して異なるタスクのアーキテクチャをロードできます．

In [8]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("distilbert/distilbert-base-uncased")

事前学習済みのモデルのインスタンスをロードするのに`AutoTokenizer`クラスと`AutoModelFor?`クラスの使用が推奨されます．
これにより，常に正しいアーキテクチャをロードできます．